In [ ]:
# !pip install transformers, tensorflow, torch

In [16]:
import warnings
from transformers import (
    AutoTokenizer, 
    BertForSequenceClassification,
    pipeline,
)

warnings.filterwarnings("ignore")

finbert_pt_br_tokenizer = AutoTokenizer.from_pretrained("lucas-leme/FinBERT-PT-BR")
finbert_pt_br_model = BertForSequenceClassification.from_pretrained("lucas-leme/FinBERT-PT-BR")

finbert_pt_br_pipeline = pipeline(task='text-classification', model=finbert_pt_br_model, tokenizer=finbert_pt_br_tokenizer)

In [15]:
import warnings
from transformers import (
    BertTokenizer, 
    BertForSequenceClassification,
    pipeline,
)

warnings.filterwarnings("ignore")

finbert_tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
finbert_model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert")

finbert_pipeline = pipeline(task='text-classification', model=finbert_model, tokenizer=finbert_tokenizer)

In [23]:
import pandas as pd

acoes = pd.read_csv('../data/news/noticias_ações.csv')
bolsa = pd.read_csv('../data/news/noticias_bolsa de valores.csv')
ibovespa = pd.read_csv('../data/news/noticias_ibovespa.csv')
mercado = pd.read_csv('../data/news/noticias_mercado financeiro.csv')

news = pd.concat([acoes, bolsa, ibovespa, mercado], ignore_index=True)
news['Data'] = pd.to_datetime(news['Data'], format='%Y-%m-%d')
news.drop_duplicates(subset='Notícia', keep='first', inplace=True)
news = news[~news['Notícia'].isin(mercado['Notícia'])]

news_list = news['Notícia'].to_list()
news_list[:5]

['Dados de emprego nos EUA impulsionam dólar; Ibovespa fecha em queda',
 'As ações campeãs para buscar dividendos em setembro, segundo 20 carteiras \nrecomendadas',
 'Gestora passa a deter mais de 27 milhões de ações do Fleury (FLRY3) – Tempo \nReal – Estadão E-Investidor – As principais notícias do mercado financeiro',
 'Estudantes de Medicina desenvolvem ações com crianças e adolescentes em \nmedidas socioeducativas e protetivas',
 'Tendências 09/09/2024 | das principais ações da Bolsa']

In [21]:
sentiments = pd.DataFrame(finbert_pt_br_pipeline(news_list))
sentiments

,label,score
0,NEGATIVE,0.638846
1,POSITIVE,0.502162
2,POSITIVE,0.631095


In [22]:
assert len(sentiments) == len(news), "Sentiments and news must have the same length"
news['Sentimento'] = sentiments['label']
news['Score'] = sentiments['score']


mercado_sentimentos = pd.read_csv('../data/news/noticias_mercado financeiro_sentimentos.csv')
mercado_sentimentos['Data'] = pd.to_datetime(mercado_sentimentos['Data'], format='%Y-%m-%d')
sentimentos = pd.concat([mercado_sentimentos, news], ignore_index=True)
sentimentos.to_csv('../data/news/noticias_mercado financeiro_sentimentos.csv', index=False)

AssertionError: Sentiments and news must have the same length

In [6]:
pd.set_option('display.max_colwidth', None)

In [18]:
# Comparação FinBERT vs FinBERT-PT-BR

# FinBERT
finbert_outputs = pd.DataFrame(finbert_pipeline(news_list[:10]))
finbert_outputs['news'] = news_list[:10]

# FinBER-PT-BR
finbert_pt_br_outputs = pd.DataFrame(finbert_pt_br_pipeline(news_list[:10]))
finbert_pt_br_outputs['news'] = news_list[:10]

# Exibindo resultados
display(
    finbert_outputs,
    finbert_pt_br_outputs,
)

,label,score,news
0,positive,0.766157,Unipar e MELVER estão com inscrições abertas para MBA em Mercado Financeiro
1,positive,0.625554,"Mercado financeiro hoje: Vale (VALE3) em busca de parceiro, Azul (AZUL4) em \napuros e acordo da Weg (WEGE3) de R$ 630 milhões – Mercado – Estadão \nE-Investidor – As principais notícias do mercado financeiro"
2,neutral,0.848194,Briga entre X e STF pode afastar investidores do Brasil? Especialistas \ndivergem
3,neutral,0.947857,Podcast | Fechamento de Mercado - 03/09/2024
4,neutral,0.935288,Comentario semanal - Perspectivas
5,neutral,0.865317,Apagão cibernético global causa tensão no mercado financeiro
6,positive,0.652962,Confira dez tendências que o mercado financeiro deve incorporar nos \npróximos anos
7,neutral,0.850085,Mercados financeiros hoje: tom negativo em NY marca volta de feriado
8,positive,0.770710,Políticas do governo federal fazem PIB crescer acima do esperado e rumar \npara 3% ao final do ano
9,positive,0.570883,Setor financeiro ganha destaque nas indicações de ações para setembro


,label,score,news
0,NEUTRAL,0.700441,Unipar e MELVER estão com inscrições abertas para MBA em Mercado Financeiro
1,NEGATIVE,0.515265,"Mercado financeiro hoje: Vale (VALE3) em busca de parceiro, Azul (AZUL4) em \napuros e acordo da Weg (WEGE3) de R$ 630 milhões – Mercado – Estadão \nE-Investidor – As principais notícias do mercado financeiro"
2,NEGATIVE,0.850306,Briga entre X e STF pode afastar investidores do Brasil? Especialistas \ndivergem
3,NEUTRAL,0.614209,Podcast | Fechamento de Mercado - 03/09/2024
4,NEUTRAL,0.610426,Comentario semanal - Perspectivas
5,NEGATIVE,0.848840,Apagão cibernético global causa tensão no mercado financeiro
6,NEUTRAL,0.725723,Confira dez tendências que o mercado financeiro deve incorporar nos \npróximos anos
7,NEGATIVE,0.821666,Mercados financeiros hoje: tom negativo em NY marca volta de feriado
8,NEGATIVE,0.554580,Políticas do governo federal fazem PIB crescer acima do esperado e rumar \npara 3% ao final do ano
9,POSITIVE,0.519682,Setor financeiro ganha destaque nas indicações de ações para setembro
